<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/basics/4_create_articulations.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. You can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Basics Tutorial 4: Create Articulations

The **articulation** is a set of **links** (each of which behaves like a rigid body) connected together with special **joints**. For instance, a drawer can be connected to a table by a **prismatic** joint (slider), and a door can be connected to a frame by a **revolute** joint (hinge). A robot is also an instance of an articulation. Articulations are usually loaded from [URDF](http://wiki.ros.org/urdf/XML) XML, which you can find example in the comprehensive tutorial linked at the top. This tutorial showcases how to create an articulation programmatically by SAPIEN.

Articulations can be controlled by applying `torques` on their joints. To drive an articulation to a desired state, users can apply a [controller](https://en.wikipedia.org/wiki/Control_theory) to compute corrective torques according to the difference between the actual and the desired.

In this tutorial, you will learn the following:
- Create Articulation
- Control the articulation with builtin controllers
- Get kinematic quantities of the articulation

The example illustrates how to build a controllable toy car from scratch.

## Preparation

> Note: you need GPU runtime to run the notebook.

In [1]:
%pip install sapien

import sapien.core as sapien
import numpy as np
from transforms3d.euler import euler2quat

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 95.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:37: UserWarning: Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.
  warn("Failed to find system libvulkan. Fallback to SAPIEN builtin libvulkan.")
/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:73: UserWarning: Failed to find Vulkan ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(
/usr/local/lib/python3.12/dist-packages/sapien/_vulkan_tricks.py:95: UserWarning: Failed to find glvnd ICD file. This is probably due to an incorrect or partial installation of the NVIDIA driver. SAPIEN will attempt to provide an ICD file anyway but it may not work.
  warn(


In [3]:
# --- 1. 超参数设置 ---
static_friction = 10.0  # 静态摩擦力
dynamic_friction = 10.0 # 动态摩擦力
restitution = 0.1      # 恢复系数（碰撞弹性）
joint_friction = 0.0    # 关节摩擦力
joint_damping = 0.0     # 关节阻尼

# --- 2. 创建场景 (SAPIEN 3.0 简化写法) ---
# 3.0 不再需要手动创建 Engine 和 Renderer，直接调用 sapien.Scene() 即可
scene = sapien.Scene()
scene.set_timestep(1 / 100.0)

# --- 3. 处理物理材质 (核心修改点) ---
# SAPIEN 3.0 的 SceneConfig 不再支持 default_static_friction 等属性
# 正确做法是创建一个 PhysicalMaterial，稍后在创建地面或小车时传入它
physical_material = scene.create_physical_material(
    static_friction=static_friction,
    dynamic_friction=dynamic_friction,
    restitution=restitution
)

# --- 4. 设置灯光 (建议在此初始化，否则渲染是黑的) ---
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])

print("SAPIEN 3.0 场景初始化成功！")

SAPIEN 3.0 场景初始化成功！


## Create a root link

In SAPIEN, the articulation is represented as a tree. Each node is a link, and each edge indicates that the child link is connected to the parent link by a joint. To build a toy car, let’s start with the car body.

    def create_car(
            scene: sapien.Scene,
            body_size=(1.0, 0.5, 0.25),
            tire_radius=0.15,
            joint_friction=0.0,
            joint_damping=0.0,
            density=1.0,
    ) -> sapien.Articulation:
        body_half_size = np.array(body_size) / 2
        shaft_half_size = np.array([tire_radius * 0.1, tire_radius * 0.1, body_size[2] * 0.1])
        rack_half_size = np.array([tire_radius * 0.1, body_half_size[1] * 2.0, tire_radius * 0.1])
        builder: sapien.ArticulationBuilder = scene.create_articulation_builder()

        # car body (root of the articulation)
        body: sapien.LinkBuilder = builder.create_link_builder()  # LinkBuilder is similar to ActorBuilder
        body.set_name('body')
        body.add_box_collision(half_size=body_half_size, density=density)
        body.add_box_visual(half_size=body_half_size, color=[0.8, 0.6, 0.4])

`Articulation` is created by `ArticulationBuilder`. Each link is built by `LinkBuilder`, which can be created by an articulation builder. A link is just a rigid body, and thus collision and visual shapes can be added. A root link is created when `create_link_builder()` is called without specifying the parent link.

## Create a child link connected by a revolute joint

Next, we create a child link (front steering shaft) connected to the root link (car body) by a revolute joint.

        # front steering shaft
        front_shaft = builder.create_link_builder(body)
        front_shaft.set_name('front_shaft')
        front_shaft.set_joint_name('front_shaft_joint')
        front_shaft.add_box_collision(half_size=shaft_half_size, density=density)
        front_shaft.add_box_visual(half_size=shaft_half_size, color=[0.6, 0.4, 0.8])
        # The x-axis of the joint frame is the rotation axis of a revolute joint.
        front_shaft.set_joint_properties(
            'revolute',
            limits=[[-np.deg2rad(15), np.deg2rad(15)]],  # joint limits (for each DoF)
            # pose_in_parent refers to the relative transformation from the parent frame to the joint frame
            pose_in_parent=sapien.Pose(
                p=[(body_half_size[0] - tire_radius), 0, -body_half_size[2]],
                q=euler2quat(0, -np.deg2rad(90), 0)
            ),
            # pose_in_child refers to the relative transformation from the child frame to the joint frame
            pose_in_child=sapien.Pose(
                p=[0.0, 0.0, shaft_half_size[2]],
                q=euler2quat(0, -np.deg2rad(90), 0)
            ),
            friction=joint_friction,
            damping=joint_damping,
        )

A child link is created when `create_link_builder(parent_link)` is called with specifying the parent link. Besides, we need to configure the joint.

There are multiple types of joints: prismatic, revolute, fixed. The definitions follow [PhysX](https://gameworksdocs.nvidia.com/PhysX/4.1/documentation/physxguide/Manual/Joints.html).

- **revolute**: a revolute joint (also called a hinge) keeps the origins and x-axes of the frames together, and allows free rotation around this common axis.
- **prismatic**: a prismatic joint (also called a slider) keeps the orientations identical, but allows the origin of each frame to slide freely along the common x-axis.
- **fixed**: a fixed joint locks the orientations and origins rigidly together

The location of the joint is defined by the joint pose in the parent frame `pose_in_parent`, and the joint pose in the child frame `pose_in_child`.

Other properties of a joint, like joint friction and joint damping, can also be set through `set_joint_properties(...)`.

Similarly we can create other necessary articulations of a toy car. The full code for building the car is provided as follow.

In [13]:
def create_car(
        scene: sapien.Scene,
        body_size=(1.0, 0.5, 0.25),
        tire_radius=0.15,
        joint_friction=0.0,
        joint_damping=0.0,
        density=1000.0, # SAPIEN 3.0 建议使用标准密度（如水为1000）
) -> sapien.Entity:
    body_half_size = np.array(body_size) / 2
    shaft_half_size = np.array([tire_radius * 0.1, tire_radius * 0.1, body_size[2] * 0.1])
    rack_half_size = np.array([tire_radius * 0.1, body_half_size[1] * 2.0, tire_radius * 0.1])

    # 获取关节体生成器
    builder = scene.create_articulation_builder()

    # 辅助函数：在 SAPIEN 3.0 中设置颜色需要创建 RenderMaterial
    def create_ant_material(color):
        mat = sapien.render.RenderMaterial()
        mat.set_base_color(np.array(color + [1.0])) # RGBA
        return mat

    # 1. 车身 (Root Link)
    body = builder.create_link_builder()
    body.set_name('body')
    body.add_box_collision(half_size=body_half_size, density=density)
    body.add_box_visual(half_size=body_half_size, material=create_ant_material([0.8, 0.6, 0.4]))

    # 2. 前转向轴 (Front steering shaft)
    front_shaft = builder.create_link_builder(body)
    front_shaft.set_name('front_shaft')
    front_shaft.set_joint_name('front_shaft_joint') # <--- 修复：必须设置唯一的关节名称
    front_shaft.add_box_collision(half_size=shaft_half_size, density=density)
    front_shaft.add_box_visual(half_size=shaft_half_size, material=create_ant_material([0.6, 0.4, 0.8]))
    front_shaft.set_joint_properties(
        'revolute',
        limits=[[-np.deg2rad(15), np.deg2rad(15)]],
        pose_in_parent=sapien.Pose(
            p=[(body_half_size[0] - tire_radius), 0, -body_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, shaft_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    # 3. 后轴 (Back steering shaft - 固定)
    back_shaft = builder.create_link_builder(body)
    back_shaft.set_name('back_shaft')
    back_shaft.set_joint_name('back_shaft_joint') # <--- 修复：必须设置唯一的关节名称
    back_shaft.add_box_collision(half_size=shaft_half_size, density=density)
    back_shaft.add_box_visual(half_size=shaft_half_size, material=create_ant_material([0.6, 0.4, 0.8]))
    back_shaft.set_joint_properties(
        'fixed',
        limits=[],
        pose_in_parent=sapien.Pose(
            p=[-(body_half_size[0] - tire_radius), 0, -body_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, shaft_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    # 4. 前轮 (Front wheels)
    front_wheels = builder.create_link_builder(front_shaft)
    front_wheels.set_name('front_wheels')
    front_wheels.set_joint_name('front_gear') # <--- 修复：必须设置唯一的关节名称
    front_wheels.add_box_collision(half_size=rack_half_size, density=density)
    front_wheels.add_box_visual(half_size=rack_half_size, material=create_ant_material([0.8, 0.4, 0.6]))

    # 左右轮
    for offset in [rack_half_size[1] + tire_radius, -(rack_half_size[1] + tire_radius)]:
        front_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, offset, 0.0]),
                                          radius=tire_radius, density=density)
        front_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, offset, 0.0]),
                                       radius=tire_radius, material=create_ant_material([0.4, 0.6, 0.8]))

    front_wheels.set_joint_properties(
        'revolute',
        limits=[[-np.inf, np.inf]],
        pose_in_parent=sapien.Pose(p=[0.0, 0, -(shaft_half_size[2] + rack_half_size[2])], q=euler2quat(0, 0, np.deg2rad(90))),
        pose_in_child=sapien.Pose(p=[0.0, 0.0, 0.0], q=euler2quat(0, 0, np.deg2rad(90))),
        friction=joint_friction,
        damping=joint_damping,
    )

    # 5. 后轮 (Back wheels)
    back_wheels = builder.create_link_builder(back_shaft)
    back_wheels.set_name('back_wheels')
    back_wheels.set_joint_name('back_gear') # <--- 修复：必须设置唯一的关节名称
    back_wheels.add_box_collision(half_size=rack_half_size, density=density)
    back_wheels.add_box_visual(half_size=rack_half_size, material=create_ant_material([0.8, 0.4, 0.6]))

    # 左右后轮
    for y_offset in [rack_half_size[1] + tire_radius, -(rack_half_size[1] + tire_radius)]:
        back_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, y_offset, 0.0]), radius=tire_radius, density=density)
        back_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, y_offset, 0.0]), radius=tire_radius, material=create_ant_material([0.4, 0.6, 0.8]))

    back_wheels.set_joint_properties(
        'revolute',
        limits=[[-np.inf, np.inf]],
        pose_in_parent=sapien.Pose(p=[0.0, 0, -(shaft_half_size[2] + rack_half_size[2])], q=euler2quat(0, 0, np.deg2rad(90))),
        pose_in_child=sapien.Pose(p=[0.0, 0.0, 0.0], q=euler2quat(0, 0, np.deg2rad(90))),
        friction=joint_friction,
        damping=joint_damping,
    )

    # SAPIEN 3.0 修正：build() 返回组件，需通过 .entity 获取实体
    articulation = builder.build()
    car_entity = articulation.entity
    car_entity.name = 'car'
    return car_entity

In [14]:
scene.add_ground(altitude=0)
car = create_car(scene, joint_friction=joint_friction, joint_damping=joint_damping)
car.set_pose(sapien.Pose(p=[0., 0., 0.34]))
print('The dof of the articulation is', car.dof)

The dof of the articulation is 3


## Control an articulation with builtin drives

After building the articulation, we want to control it by actuating its joints. SAPIEN provides builtin **drives** (controllers) to control either the position or the speed of a joint.

In [10]:
def get_joints_dict(articulation_handle):
    # SAPIEN 3.0 兼容逻辑：
    # 如果传入的是 Entity，则获取其物理组件
    if hasattr(articulation_handle, "get_component"):
        articulation_comp = articulation_handle.get_component(sapien.physx.PhysxArticulationComponent)
    else:
        # 如果传入的已经是组件（builder.build() 的返回值），直接使用
        articulation_comp = articulation_handle

    if not articulation_comp:
        print("错误：无法获取关节体组件！")
        return {}

    # 获取关节
    joints = articulation_comp.get_joints()
    joint_names = [joint.name for joint in joints]

    assert len(joint_names) == len(set(joint_names)), 'Joint names are assumed to be unique.'
    return {joint.name: joint for joint in joints}

All the joints of an articulation can be acquired by `get_joints()`.

> Note: Although the order of joints returned by `get_joints()` is fixed, it is recommended to index a joint by its name. Joint names should be unique, which is not forced in SAPIEN though.

In [11]:
joints = get_joints_dict(car)
print(joints.keys())
# 接下来的设置代码现在可以正常执行了
joints['front_shaft_joint'].set_drive_property(stiffness=1000.0, damping=100.0)

AssertionError: Joint names are assumed to be unique.

For each active joint (with non-zero degree of freedom), we can set its drive properties: `stiffness` and `damping`. They implies the extent to which the drive attempts to achieve the target position and velocity respectively. There do not exist a general rule to set those values and you usually should tune them case by case. If you are familiar with control theory, they correspond to P and D terms in [PID controller](https://en.wikipedia.org/wiki/PID_controller). The initial target position and velocity of a joint are zero by default. Since our toy car is designed to be a front-wheel drive car, we set both the stiffness and damping as zero for the back gear.

> When a non-zero target is set and stiffness/damping is also non-zero, the drive takes effect internally at each simulation step.

## Remove an articulation

Similar to removing an actor, `scene.remove_articulation(articulation)` will remove it from the scene. Using the articulation or any of its links or joints after removal will result in undefined behavior (usually a crash).


## Get kinematic quantities of the articulation

The pose of the articulation (frame) in the world frame can be acquired by `get_pose()`. It is the same as the pose of the root link. Besides, joint positions and velocities can be acquired by `get_qpos()` and `get_qvel()`. They both return a list of scalars, the length of which is the total degree of freedom. The order is the same as `get_joints()`.

If running locally, we can use `Viewer` to visualize the toy car. We can implement different behaviors when different keys are pressed. `set_drive_target(...)` and `set_drive_velocity_target(...)` are called to set the target position and velocity of a joint drive.

In [ ]:
if not IN_COLAB:
    from sapien.utils.viewer import Viewer

    # Set up viewer
    viewer = Viewer(renderer)
    viewer.set_scene(scene)
    viewer.set_camera_xyz(x=-12, y=0, z=15)
    viewer.set_camera_rpy(r=0, p=-np.arctan2(2, 2), y=0)
    viewer.window.set_camera_parameters(near=0.05, far=100, fovy=1)
    viewer.focus_entity(car)

    # Set up light
    scene.set_ambient_light([0.5, 0.5, 0.5])
    scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5], shadow=True)

    position = 0.0  # position target of joints
    velocity = 0.0  # velocity target of joints
    steps = 0
    last_step = -4
    while not viewer.closed:
        if steps - last_step < 4:
            pass  # prevent multiple changes for one key press
        else:
            last_step = steps
            if viewer.window.key_down('i'):  # accelerate
                velocity += 5.0
                print('velocity increases:', velocity)
                joints['front_gear'].set_drive_velocity_target(velocity)
            elif viewer.window.key_down('k'):  # brake
                velocity -= 5.0
                print('velocity decreases:', velocity)
                joints['front_gear'].set_drive_velocity_target(velocity)
            elif viewer.window.key_down('j'):  # left turn
                position += 2
                position = np.clip(position, *limits)
                joints['front_shaft_joint'].set_drive_target(np.deg2rad(position))
                print('position increases:', position)
            elif viewer.window.key_down('l'):  # right turn
                position -= 2
                position = np.clip(position, *limits)
                print('position decreases:', position)
                joints['front_shaft_joint'].set_drive_target(np.deg2rad(position))
            elif viewer.window.key_down('r'):  # reset
                position = 0
                velocity = 0.0
                print('reset')
                joints['front_shaft_joint'].set_drive_target(position)
                joints['front_gear'].set_drive_velocity_target(velocity)

        car.set_qf(car.compute_passive_force(True, True, False))
        scene.step()
        scene.update_render()
        viewer.render()

        if steps % 500 == 0:
            print('step:', steps)
            print('Pose', car.get_pose())
            print('Joint positions', car.get_qpos())
            print('Joint velocities', car.get_qvel())
        steps += 1